In [6]:
import pandas as pd
# Verificar si el archivo existe
file_path = '../Limpieza/data_dummies.csv'

data = pd.read_csv(file_path, sep=';')
 
data.head()


,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown,y_yes
0,58,2143,5,261,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,44,29,5,151,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,33,2,5,76,1,-1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
3,47,1506,5,92,1,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
4,33,1,5,198,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
data.shape

(45211, 43)

In [8]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# Separar características (X) y variable objetivo (y)
X = data.drop(columns=['y_yes'])
y = data['y_yes']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características (Lasso es sensible a la escala de las variables)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y ajustar el modelo Lasso
lasso = Lasso(alpha=0.1)  # Ajusta alpha para controlar la regularización
lasso.fit(X_train_scaled, y_train)

# Imprimir las características seleccionadas (con coeficiente distinto de cero)
selected_features = [feature for feature, coef in zip(X.columns, lasso.coef_) if coef != 0]
print("Características seleccionadas:", selected_features)

# Evaluar el modelo Lasso
y_pred = lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Coeficiente de determinación R^2:", lasso.score(X_test_scaled, y_test))


Características seleccionadas: ['duration']
Mean Squared Error: 0.09996054524618525
Coeficiente de determinación R^2: 0.05777948465198668


In [9]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Separar características (X) y variable objetivo (y)
X = data.drop(columns=['y_yes'])  # Características
y = data['y_yes']  # Variable objetivo

# Crear un pipeline que incluya el escalado y el modelo Lasso
pipeline = make_pipeline(StandardScaler(), Lasso(alpha=0.1))

# Definir el objeto KFold para 5 particiones (por ejemplo)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Realizar la validación cruzada
scores = cross_val_score(pipeline, X, y, cv=kf, scoring='r2')

# Mostrar los resultados
print("Cross-validation R^2 scores:", scores)
print("Mean Cross-validation R^2 score:", scores.mean())




Cross-validation R^2 scores: [0.05988667 0.05702819 0.05630836 0.05735894 0.05613545 0.05672125
 0.05744418 0.05903862 0.06295517 0.06212107]
Mean Cross-validation R^2 score: 0.05849978976333855


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# Crear el modelo de regresión logística
logistic_regression = LogisticRegression()

# Realizar la validación cruzada con la métrica de precisión (accuracy)
scores = cross_val_score(logistic_regression, X, y, cv=10, scoring='accuracy')

# Mostrar los resultados
print("Cross-validation accuracy scores:", scores)
print("Mean cross-validation accuracy:", scores.mean())


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Cross-validation accuracy scores: [0.90468819 0.90776377 0.9048883  0.87856669 0.88608715 0.88365406
 0.85733245 0.87303694 0.88564477 0.85202389]
Mean cross-validation accuracy: 0.8833686200355881


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:

# Asegurarse de que el modelo esté ajustado antes de acceder a coef_
logistic_regression.fit(X_train, y_train)

# Obtener el coeficiente para cada característica
coef = logistic_regression.coef_[0]  # .coef_ es una matriz de 2D; [0] selecciona la primera fila
variables = X.columns

# Imprimir cada variable con su coeficiente
for var, c in zip(variables, coef):
    print(f"Variable: {var}, Coeficiente: {c}")

Variable: age, Coeficiente: -0.01703285836807898
Variable: balance, Coeficiente: 1.7631351319968e-05
Variable: day, Coeficiente: -0.010300898319114715
Variable: duration, Coeficiente: 0.0038234833888144575
Variable: campaign, Coeficiente: -0.3844342717945095
Variable: pdays, Coeficiente: 0.0023113371798994154
Variable: previous, Coeficiente: -0.14579819447206896
Variable: job_blue-collar, Coeficiente: -0.2884440152805934
Variable: job_entrepreneur, Coeficiente: -0.027931949303613538
Variable: job_housemaid, Coeficiente: -0.00605384785894319
Variable: job_management, Coeficiente: 0.007488035283181514
Variable: job_retired, Coeficiente: 0.182535413423583
Variable: job_self-employed, Coeficiente: -0.014380571015067544
Variable: job_services, Coeficiente: -0.09613022347680289
Variable: job_student, Coeficiente: 0.028846413772819376
Variable: job_technician, Coeficiente: -0.08324617623427633
Variable: job_unemployed, Coeficiente: 0.005016086284404601
Variable: job_unknown, Coeficiente: 0.00

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Función para entrenar un modelo de regresión logística y calcular el accuracy
def train_and_evaluate_logit(X_train, X_test, y_train, y_test):
    # Agregar constante al conjunto de entrenamiento
    X_train_const = sm.add_constant(X_train)
    
    # Crear y ajustar el modelo de regresión logística
    model = sm.Logit(y_train, X_train_const)
    result = model.fit(disp=False)  # disp=False oculta los detalles de ajuste en cada iteración
    
    # Agregar constante al conjunto de prueba y calcular el accuracy
    X_test_const = sm.add_constant(X_test, has_constant='add')
    y_pred_prob = result.predict(X_test_const)
    y_pred = [1 if p >= 0.5 else 0 for p in y_pred_prob]
    accuracy = accuracy_score(y_test, y_pred)
    
    return result, accuracy

# Entrenar el modelo inicial para obtener los valores p
initial_result, initial_accuracy = train_and_evaluate_logit(X_train, X_test, y_train, y_test)
print("Initial accuracy on test set:", initial_accuracy)

# Obtener las variables significativas (p < 0.05) en el modelo entrenado
significant_vars = initial_result.pvalues[initial_result.pvalues < 0.05].index
significant_vars = significant_vars.drop('const', errors='ignore')  # Excluir la constante si está presente

# Filtrar el conjunto de datos de entrenamiento y prueba con las variables significativas
X_train_significant = X_train[significant_vars]
X_test_significant = X_test[significant_vars]

# Reentrenar el modelo con las variables significativas
final_result, final_accuracy = train_and_evaluate_logit(X_train_significant, X_test_significant, y_train, y_test)
print("\nFinal model summary:")
print(final_result.summary())
print("\nFinal accuracy on test set:", final_accuracy)


Initial accuracy on test set: 0.8987061815769103

Final model summary:
                           Logit Regression Results                           
Dep. Variable:                  y_yes   No. Observations:                36168
Model:                          Logit   Df Residuals:                    36138
Method:                           MLE   Df Model:                           29
Date:                Fri, 01 Nov 2024   Pseudo R-squ.:                  0.3374
Time:                        17:35:53   Log-Likelihood:                -8603.3
converged:                       True   LL-Null:                       -12985.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -2.4412      0.111    -22.092      0.000      -2.658      -2.225
day                     0.0101   

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

# Cargar el archivo
file_path = '../Limpieza/data_dummies.csv'
data = pd.read_csv(file_path, sep=';')

# Separar variables predictoras y variable de interés
X = data.drop(columns=['y_yes'])
y = data['y_yes']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear un pipeline con estandarización y regresión logística con Elastic Net
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Estandarización de datos
    ('logreg', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000,class_weight='balanced'))  # Elastic Net
])

# Definir los hiperparámetros para Elastic Net
param_grid = {
    'logreg__l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Peso entre L1 y L2 (Elastic Net)
    'logreg__C': [0.01, 0.1, 1, 10, 100]  # Inverso de la regularización
}

# Búsqueda de hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Resultados del mejor modelo
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor AUC en validación:", grid_search.best_score_)

# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("AUC en el conjunto de prueba:", roc_auc_score(y_test, y_pred_proba))


Mejores hiperparámetros: {'logreg__C': 0.01, 'logreg__l1_ratio': 0.5}
Mejor AUC en validación: 0.9097856062269185
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.97      0.85      0.91     11966
           1       0.42      0.82      0.56      1598

    accuracy                           0.85     13564
   macro avg       0.70      0.84      0.73     13564
weighted avg       0.91      0.85      0.87     13564

AUC en el conjunto de prueba: 0.9097857467246057


In [14]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd

# Cargar el archivo
file_path = '../Limpieza/data_dummies.csv'
data = pd.read_csv(file_path, sep=';')

# Separar variables predictoras y variable de interés
X = data.drop(columns=['y_yes'])
y = data['y_yes']

# Aplicar SMOTE después de dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear un pipeline con estandarización y regresión logística con Elastic Net
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Estandarización de datos
    ('logreg', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000))  # Elastic Net
])

# Definir los hiperparámetros para Elastic Net
param_grid = {
    'logreg__l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Peso entre L1 y L2 (Elastic Net)
    'logreg__C': [0.01, 0.1, 1, 10, 100]  # Inverso de la regularización
}

# Búsqueda de hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# Resultados del mejor modelo
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor AUC en validación:", grid_search.best_score_)

# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("AUC en el conjunto de prueba:", roc_auc_score(y_test, y_pred_proba))


Mejores hiperparámetros: {'logreg__C': 0.01, 'logreg__l1_ratio': 0.9}
Mejor AUC en validación: 0.9676247398811582
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.93      0.94      0.93      7952
           1       0.52      0.47      0.49      1091

    accuracy                           0.88      9043
   macro avg       0.72      0.70      0.71      9043
weighted avg       0.88      0.88      0.88      9043

AUC en el conjunto de prueba: 0.8705527159289376


In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.cross_decomposition import PLSRegression
import pandas as pd

# Cargar los datos
file_path = '../Limpieza/data_dummies.csv'
data = pd.read_csv(file_path, sep=';')

# Separar variables predictoras y variable de interés
X = data.drop(columns=['y_yes'])
y = data['y_yes']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reducción de dimensionalidad con PLS (si es necesario)
pls = PLSRegression(n_components=5)  # Puedes ajustar el número de componentes
X_train_pls = pls.fit_transform(X_train, y_train)[0]
X_test_pls = pls.transform(X_test)

# Implementación de LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_pls, y_train)

# Predicción y evaluación
y_pred = lda.predict(X_test_pls)
print("LDA Classification Report:\n", classification_report(y_test, y_pred))


LDA Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94     11966
           1       0.60      0.43      0.50      1598

    accuracy                           0.90     13564
   macro avg       0.76      0.69      0.72     13564
weighted avg       0.89      0.90      0.89     13564



In [16]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Implementación de QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_pls, y_train)

# Predicción y evaluación
y_pred_qda = qda.predict(X_test_pls)
print("QDA Classification Report:\n", classification_report(y_test, y_pred_qda))


QDA Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94     11966
           1       0.57      0.46      0.51      1598

    accuracy                           0.89     13564
   macro avg       0.75      0.71      0.72     13564
weighted avg       0.89      0.89      0.89     13564



In [17]:
# Contar el número de valores en uno
count_ones = y.sum()

# Calcular la proporción
proportion_ones = count_ones / len(y)

print(f"Cantidad de 'y_yes' en uno: {count_ones}")
print(f"Proporción de 'y_yes' en uno: {proportion_ones:.2%}")

Cantidad de 'y_yes' en uno: 5289
Proporción de 'y_yes' en uno: 11.70%


In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.cross_decomposition import PLSRegression
import pandas as pd

# Cargar los datos
file_path = '../Limpieza/data_dummies.csv'
data = pd.read_csv(file_path, sep=';')

# Separar variables predictoras y variable de interés
X = data.drop(columns=['y_yes'])
y = data['y_yes']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reducción de dimensionalidad con PLS (si es necesario)
pls = PLSRegression(n_components=5)  # Puedes ajustar el número de componentes
X_train_pls = pls.fit_transform(X_train, y_train)[0]
X_test_pls = pls.transform(X_test)

# Ajuste de pesos para LDA
priors = [0.88, 0.12]  # Puedes ajustar los pesos en función de la proporción de las clases
lda = LinearDiscriminantAnalysis(priors=priors)
lda.fit(X_train_pls, y_train)

# Predicción y evaluación
y_pred = lda.predict(X_test_pls)
print("LDA Classification Report:\n", classification_report(y_test, y_pred))
# Coeficientes en el espacio reducido (transformado por PLS)
print("Coeficientes de LDA en el espacio reducido:\n", lda.coef_)



LDA Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94     11966
           1       0.59      0.43      0.50      1598

    accuracy                           0.90     13564
   macro avg       0.76      0.70      0.72     13564
weighted avg       0.89      0.90      0.89     13564

Coeficientes de LDA en el espacio reducido:
 [[ 1.54354385  0.84428107  0.27205597 -0.15823337 -0.12755082]]


In [19]:
import numpy as np

# Obtener los coeficientes de LDA en el espacio reducido
lda_coef_reduced = lda.coef_

# Coeficientes de PLS que relacionan los componentes con las variables originales
pls_coef = pls.x_weights_

# Aproximación de los coeficientes en el espacio original
# Multiplicamos los coeficientes de LDA por los coeficientes de PLS
coef_original_space = np.dot(pls_coef, lda_coef_reduced.T)

# Mostrar los coeficientes aproximados en el espacio original
print("Coeficientes aproximados de LDA en el espacio original:\n", coef_original_space)


Coeficientes aproximados de LDA en el espacio original:
 [[ 1.07618304e-02]
 [ 1.93422510e-02]
 [ 7.70853168e-02]
 [ 1.37405646e+00]
 [-5.05569767e-02]
 [-9.03371794e-02]
 [-2.11508221e-02]
 [-3.75523268e-02]
 [-3.33979222e-02]
 [-4.23744493e-02]
 [-4.85979621e-02]
 [ 7.43989756e-02]
 [-3.61186453e-02]
 [-1.65612547e-02]
 [ 1.10770821e-01]
 [-3.63268140e-02]
 [ 3.50182135e-04]
 [-2.84252201e-02]
 [-6.95573600e-02]
 [ 6.84306031e-02]
 [ 7.19959343e-02]
 [ 5.34420746e-02]
 [ 1.27719915e-02]
 [ 5.95854364e-04]
 [-2.48237948e-01]
 [-1.01429334e-01]
 [-2.37401942e-02]
 [-3.31928598e-01]
 [-2.10058088e-01]
 [ 1.08266822e-01]
 [-1.44910539e-02]
 [-1.80651251e-01]
 [-2.17016903e-01]
 [ 8.96206712e-02]
 [ 3.25603312e-01]
 [-8.62638510e-02]
 [-1.76584242e-01]
 [ 2.59833283e-01]
 [ 2.24405932e-01]
 [ 3.08250655e-02]
 [ 8.15650967e-01]
 [-6.24098667e-02]]


In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.cross_decomposition import PLSRegression
import numpy as np
import pandas as pd
from sklearn.utils import resample

# Cargar los datos
file_path = '../Limpieza/data_dummies.csv'
data = pd.read_csv(file_path, sep=';')

# Separar variables predictoras y variable de interés
X = data.drop(columns=['y_yes'])
y = data['y_yes']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reducción de dimensionalidad con PLS (si es necesario)
pls = PLSRegression(n_components=5)  # Puedes ajustar el número de componentes
X_train_pls = pls.fit_transform(X_train, y_train)[0]
X_test_pls = pls.transform(X_test)

# Ajuste de pesos para LDA
priors = [0.88, 0.12]  # Puedes ajustar los pesos en función de la proporción de las clases
lda = LinearDiscriminantAnalysis(priors=priors)
lda.fit(X_train_pls, y_train)

# Predicción y evaluación
y_pred = lda.predict(X_test_pls)
print("LDA Classification Report:\n", classification_report(y_test, y_pred))

# Coeficientes en el espacio reducido (transformado por PLS)
lda_coef_reduced = lda.coef_

# Coeficientes de PLS que relacionan los componentes con las variables originales
pls_coef = pls.x_weights_

# Aproximación de los coeficientes en el espacio original
coef_original_space = np.dot(pls_coef, lda_coef_reduced.T)
print("Coeficientes aproximados de LDA en el espacio original:\n", coef_original_space)

# -------------------------------------------------
# Intervalos de Confianza mediante Bootstrap
# -------------------------------------------------
n_bootstrap = 1000
coef_values_bootstrap = []

for i in range(n_bootstrap):
    # Remuestreo con reemplazo
    X_resampled, y_resampled = resample(X, y, random_state=i)
    
    # Reducción de dimensionalidad con PLS en el conjunto resampleado
    pls_resampled = PLSRegression(n_components=5)
    X_resampled_pls = pls_resampled.fit_transform(X_resampled, y_resampled)[0]
    
    # Entrenar LDA en el espacio reducido resampleado
    lda_resampled = LinearDiscriminantAnalysis(priors=[0.88, 0.12])
    lda_resampled.fit(X_resampled_pls, y_resampled)
    
    # Obtener los coeficientes aproximados en el espacio original
    coef_resampled_original = np.dot(pls_resampled.x_weights_, lda_resampled.coef_.T)
    coef_values_bootstrap.append(coef_resampled_original.flatten())

# Convertir a matriz para calcular intervalos de confianza
coef_values_bootstrap = np.array(coef_values_bootstrap)
lower_bounds = np.percentile(coef_values_bootstrap, 2.5, axis=0)
upper_bounds = np.percentile(coef_values_bootstrap, 97.5, axis=0)

print("Intervalos de confianza del 95% para los coeficientes:\n")
for i in range(len(lower_bounds)):
    print(f"Variable {i+1}: ({lower_bounds[i]:.4f}, {upper_bounds[i]:.4f})")


LDA Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94     11966
           1       0.59      0.43      0.50      1598

    accuracy                           0.90     13564
   macro avg       0.76      0.70      0.72     13564
weighted avg       0.89      0.90      0.89     13564

Coeficientes aproximados de LDA en el espacio original:
 [[ 1.07618304e-02]
 [ 1.93422510e-02]
 [ 7.70853168e-02]
 [ 1.37405646e+00]
 [-5.05569767e-02]
 [-9.03371794e-02]
 [-2.11508221e-02]
 [-3.75523268e-02]
 [-3.33979222e-02]
 [-4.23744493e-02]
 [-4.85979621e-02]
 [ 7.43989756e-02]
 [-3.61186453e-02]
 [-1.65612547e-02]
 [ 1.10770821e-01]
 [-3.63268140e-02]
 [ 3.50182135e-04]
 [-2.84252201e-02]
 [-6.95573600e-02]
 [ 6.84306031e-02]
 [ 7.19959343e-02]
 [ 5.34420746e-02]
 [ 1.27719915e-02]
 [ 5.95854364e-04]
 [-2.48237948e-01]
 [-1.01429334e-01]
 [-2.37401942e-02]
 [-3.31928598e-01]
 [-2.10058088e-01]
 [ 1.08266822e-01]
 [-1.44910539e-02]

In [39]:
import numpy as np
import pandas as pd

# Valores válidos extraídos del diccionario
valores_validos = {
    "job": ["admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student",
            "blue-collar", "self-employed", "retired", "technician", "services"],
    "marital": ["married", "divorced", "single"],
    "education": ["unknown", "secondary", "primary", "tertiary"],
    "default": ["yes", "no"],
    "housing": ["yes", "no"],
    "loan": ["yes", "no"],
    "contact": ["unknown", "telephone", "cellular"],
    "month": ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"],
    "poutcome": ["unknown", "other", "failure", "success"]
}

# Rango razonable para variables numéricas
rangos_numericos = {
    "age": (18, 100),
    "balance": (-10000, 100000),
    "day": (1, 31),
    "duration": (0, 5000),  # segundos
    "campaign": (1, 50),
    "pdays": (-1, 500),  # -1 significa que no fue contactado previamente
    "previous": (0, 100)
}

def predecir_cliente_v2(age, balance, day, duration, campaign, pdays, previous, job=None, marital=None,
                        education=None, default=None, housing=None, loan=None, contact=None, month=None,
                        poutcome=None, pls_modelo=None, lda_modelo=None, columnas_X=None):
    """
    Función para predecir la clase de un cliente dado sus características.
    """

    # Crear un DataFrame vacío con las mismas columnas que el conjunto de entrenamiento X
    cliente_df = pd.DataFrame(columns=columnas_X)
    cliente_df.loc[0] = 0  # Inicializar todas las columnas en 0

    # Validar y asignar valores numéricos
    for var, rango in rangos_numericos.items():
        valor = locals()[var]
        if not (rango[0] <= valor <= rango[1]):
            raise ValueError(f"El valor de {var} ({valor}) está fuera del rango {rango}.")
        cliente_df.loc[0, var] = valor

    # Validar y asignar valores categóricos
    for var, categorias in valores_validos.items():
        valor = locals()[var]
        if valor is not None:
            if valor not in categorias:
                raise ValueError(f"Valor '{valor}' no válido para {var}. Debe ser uno de {categorias}.")
            # Activar solo la columna correspondiente si está en X
            col_name = f"{var}_{valor}"
            if col_name in cliente_df.columns:
                cliente_df.loc[0, col_name] = 1

    # Convertir todos los valores a float
    cliente_df = cliente_df.astype(float)

    # Verificar que cliente_df tenga exactamente las mismas columnas y el mismo orden que X
    if list(cliente_df.columns) != list(columnas_X):
        columnas_extra = set(cliente_df.columns) - set(columnas_X)
        columnas_faltantes = set(columnas_X) - set(cliente_df.columns)
        raise ValueError(f"Las columnas de cliente_df no coinciden con las del conjunto de entrenamiento X.\n"
                         f"Columnas adicionales en cliente_df: {columnas_extra}\n"
                         f"Columnas faltantes en cliente_df: {columnas_faltantes}")

    # Transformar las características del cliente usando el modelo PLS
    cliente_pls = pls_modelo.transform(cliente_df.values)

    # Predecir la clase con el modelo LDA
    prediccion = lda_modelo.predict(cliente_pls)[0]
    probabilidad = lda_modelo.predict_proba(cliente_pls)[0]

    return prediccion, probabilidad


In [40]:
# Llamada a la función con X.columns para asegurar las columnas correctas
prediccion, probabilidad = predecir_cliente_v2(
    age=34, balance=1500, day=15, duration=200, campaign=3, pdays=5, previous=2,
    job="unemployed", marital="single", education="tertiary", default="no",
    housing="yes", loan="no", contact="cellular", month="may", poutcome="unknown",
    pls_modelo=pls, lda_modelo=lda, columnas_X=X.columns
)

print(f"Clase predicha para el cliente: {prediccion}")
print(f"Probabilidad de pertenecer a cada clase: {probabilidad}")


Clase predicha para el cliente: 0
Probabilidad de pertenecer a cada clase: [0.97923006 0.02076994]


c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PLSRegression was fitted with feature names
  warnings.warn(
